# CMSE 381 Final Project Template

**INSTRUCTIONS**: This is a template to help organize your project.  All projects should include the 5 major sections below (you do not need to use this template file).  If you use this file, complete your work below and remove content in parentheses. Also, remove this current cell.  

#### CMSE 381 Final Project
### &#9989; Group members: Kyle Landolt, Isaac Maki
### &#9989; Section_001
#### &#9989; 4/16/2025

# ___Can we classify the facial orientation provided to the monkey by measuring if the neuron fires at site 041 and site 157 during the first 400 milliseconds of the monkey being shown the image?___

## Background and Motivation

**We got this motivation by just brainstorming a few problems by checking the dataset. We looked through the data and read the research paper, and decided on this together. There would be many routes you could take in analyzing this data. The researchers analyzed from many different sites in the brains, across multiple different monkeys. However, we wanted to have a simpler question, so we narrowed our study down to two sites to make models that could possibly be applied to other sites as well. We were really intrigued by the fact that monkeys can potentially detect a person's face at certain positions better by measuring certain sites in their brains. We want to discover and see if we are able to predict the facial orientation due to the amount of neural activity spiking in the Monkeys brain within the first 400 millisecond time frame of being shown the photo.**

## Methodology
_(How did you go about answering your question(s)? You should wrote some code here to demonstrate what the data is like and how in principle your method works. You can leave the variations of the related to specific results to the results section.)_

In [1]:
import pandas as pd #This helps us read in the data
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict #This is how the k-fold is used
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay #This displays the confusion matrix after the k-fold cv
import matplotlib.pyplot as plt #This one is for plotting

#Kyle - our current plan is that I will be doing a logistic regression model and Isaac will be doing a decision tree about our research question.
#I am going to go through our old class jupyter notebooks where we worked on logistic regression and set up multiple models with our current dataset. I will
#write a little pseudocode once I get the code going.

In [2]:
data = pd.read_csv('raster_data_lupo_am_site157.csv') #Reads in dataset
data["is_front"] = (data["labels.orientation"] == "front").astype(int) #This creates a new column of just front data to train
X = data.filter(like="time") #This filters the data for all of the time columns (I had to look this one up for all the time parts through ChatGPT)
y = data["is_front"] #assigns y to our dependent variable
from sklearn.linear_model import LogisticRegression #importing LogisticRegression function from sklearn module
from sklearn.model_selection import cross_val_score #
clf = LogisticRegression(max_iter=1000)
scores = cross_val_score(clf, X, y, cv=5)
print("Mean accuracy (binary):", scores.mean())

Mean accuracy (binary): 0.8725422160536664


### Data
_(Describe the data you are using. What variables are you using? What they mean? Why did you choose them?)_

**Our dataset consists of a LOT of features, mainly we are looking at the feature "labels.orientation", which is a string that represents what
side of the face the monkey is seeing. Outside of the times, we have 5 other features: "site_info.monkey", "site_info.region"
,"labels.stimID", "labels.person" and "labels.orient_person_combo". These features are important to look at and understand. They give an understanding of how the methodology of the research was carried out. However, they don't really help much in predicting, especially compared to the time frames given. The time frame columns were given by increments of 1 millisecond, all the way to 800ms. However, we dropped all the columns from 401ms to 800ms, as the first 400ms are when the monkey is actually shown the image. Therefore, we used 400 columns, representing each of the milliseconds between 1ms to 400ms, to predict the "label.orientation" column.**

site_info.monkey: "Name of the monkey (always lupo)"
site_info.region: "Region of the brain where the data is being collected from (always am), (I looked it up, this means"
labels.stimID: "An ID number that connects the person and the type of orientation that the picture is"
labels.person: "The person who the picture is of, in this dataset it is 1 0f 25 people" 
labels.orientation: "String that represents what side of the face the monkey is being shown"
labels.orient_person_combo: "Just the orientation and then an int value that is the number in labels.person"
All time variables: "Binary value meaning either 1 or 0 where 1 represents a neural activity spike and 0 represents the other side"
#We chose orientation because a lot of these predictors are pretty irrelevant. First off, region and monkey are always the same value,
#second off, stimID, orientation_person_combo, and person arent very telling predictors, so we went with our predictor we liked the most,
#orientation

In [3]:
data = pd.read_csv('raster_data_lupo_am_site157.csv')
data.head(5)
#Here is the dataset loaded in and looked at and you can see that the predictors I described in the previous tab are all here

,site_info.monkey,site_info.region,labels.stimID,labels.person,labels.orientation,labels.orient_person_combo,time.1_2,time.2_3,time.3_4,time.4_5,...,time.791_792,time.792_793,time.793_794,time.794_795,time.795_796,time.796_797,time.797_798,time.798_799,time.799_800,time.800_801
0,lupo,am,1,1,front,front 1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,lupo,am,1,1,front,front 1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,lupo,am,1,1,front,front 1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,lupo,am,2,2,front,front 2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,lupo,am,2,2,front,front 2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Models for classification _(if applicable)_
_(What models will you be using for classification? Why did you choose to use them? What questions would you answer with them? How would you evaluate if each model? What cross-validation method did you use?)_

In [4]:
site041_data = pd.read_csv("raster_data_bert_am_site041.csv")
site041_data = site041_data.drop(site041_data.columns[405:806], axis=1)

#splitting based upon predictors (1ms - 400ms) and the label orientation
orientation = site041_data["labels.orientation"]
predictors = site041_data.drop(columns = ["site_info.monkey", "site_info.region", "labels.stimID", "labels.person", "labels.orientation", "labels.orient_person_combo"])
X_train1, X_test1, y_train1, y_test1 = train_test_split(predictors, orientation, test_size=0.2, random_state=1)

In [5]:
#creating the first, most simple tree
first_tree = tree.DecisionTreeClassifier(max_depth = 3, max_leaf_nodes=8, random_state = 1)
first_tree = first_tree.fit(X_train1, y_train1)

In [6]:
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier, plot_tree

y_pred1 = first_tree.predict(X_test1)
print(classification_report(y_test1, y_pred1))

               precision    recall  f1-score   support

         back       0.10      1.00      0.18        29
         down       0.00      0.00      0.00        44
        front       0.00      0.00      0.00        34
     left 3/4       0.00      0.00      0.00        40
 left profile       0.00      0.00      0.00        38
    right 3/4       0.00      0.00      0.00        51
right profile       0.00      0.00      0.00        36
           up       0.67      0.05      0.09        40

     accuracy                           0.10       312
    macro avg       0.10      0.13      0.03       312
 weighted avg       0.09      0.10      0.03       312



/opt/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
constant_cols = [col for col in predictors.columns if predictors[col].nunique() == 1]
predictors_filtered = predictors.drop(columns=constant_cols)

In [8]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(predictors_filtered, orientation, test_size=0.2, random_state=1)
second_tree = tree.DecisionTreeClassifier(max_depth = 3, max_leaf_nodes=8, random_state = 1)
second_tree = second_tree.fit(X_train2, y_train2)

In [9]:
y_pred2 = second_tree.predict(X_test2)
print(classification_report(y_test2, y_pred2))

               precision    recall  f1-score   support

         back       0.10      1.00      0.18        29
         down       0.00      0.00      0.00        44
        front       0.00      0.00      0.00        34
     left 3/4       0.00      0.00      0.00        40
 left profile       0.00      0.00      0.00        38
    right 3/4       0.00      0.00      0.00        51
right profile       0.00      0.00      0.00        36
           up       0.67      0.05      0.09        40

     accuracy                           0.10       312
    macro avg       0.10      0.13      0.03       312
 weighted avg       0.09      0.10      0.03       312



/opt/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [10]:
n_chunks = 8
chunk_size = 50

grouped_data = []

for i in range(n_chunks):
    start = i * chunk_size
    end = start + chunk_size
    chunk_sum = predictors.iloc[:, start:end].sum(axis=1)
    grouped_data.append(chunk_sum)

# Stack the columns side-by-side
X_grouped = pd.concat(grouped_data, axis=1)
X_grouped.columns = [f"chunk_{i+1}" for i in range(n_chunks)]


In [12]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_grouped, orientation, test_size=0.2, random_state=1)
third_tree = tree.DecisionTreeClassifier(max_depth = 3, max_leaf_nodes=8, random_state = 1)
third_tree = third_tree.fit(X_train3, y_train3)

In [14]:
y_pred3 = third_tree.predict(X_test3)
print(classification_report(y_test3, y_pred3))

               precision    recall  f1-score   support

         back       0.15      0.76      0.25        29
         down       0.00      0.00      0.00        44
        front       0.05      0.06      0.05        34
     left 3/4       0.22      0.20      0.21        40
 left profile       0.00      0.00      0.00        38
    right 3/4       0.19      0.16      0.17        51
right profile       0.00      0.00      0.00        36
           up       0.36      0.40      0.38        40

     accuracy                           0.18       312
    macro avg       0.12      0.20      0.13       312
 weighted avg       0.12      0.18      0.13       312



/opt/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Models for regression _(if applicable)_
_(What models will you be using for regression? Why did you choose to use them? What questions would you answer with them? How would you evaluate if each model? What cross-validation method did you use?)_

In [ ]:
# you may add some code here to show how the model works in principle
data = pd.read_csv('raster_data_lupo_am_site157.csv') #Reads in dataset
data = data.drop(data.columns[405:806], axis = 1)
orientations = ['right 3/4','front','left profile','left 3/4','up','back','down','right profile']
for orientation in orientations:
    data["is_front"] = (data["labels.orientation"] == orientation).astype(int) #This creates a new column of just front data to train
    X = data.filter(like="time") #This filters the data for all of the time columns (I had to look this one up for all the time parts through ChatGPT)
    y = data["is_front"] #assigns y to our dependent variable
    from sklearn.linear_model import LogisticRegression #importing LogisticRegression function from sklearn module
    from sklearn.model_selection import cross_val_score
    clf = LogisticRegression(max_iter=1000, class_weight='balanced')
    scores = cross_val_score(clf, X, y, cv=5)
    print("Mean accuracy (binary) for",orientation,":", scores.mean())

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
data = pd.read_csv('raster_data_lupo_am_site157.csv')
data = data.drop(data.columns[405:806], axis = 1)
orientations = ['right 3/4','front','left profile','left 3/4','up','back','down','right profile']
for orientation in orientations:
    print(orientation, y.value_counts())
    X = data.filter(like="time")
    y = (data["labels.orientation"] == orientation).astype(int)
    clf = LogisticRegression(max_iter=1000, class_weight='balanced')
    y_pred = cross_val_predict(clf, X, y, cv=10)
    cm = confusion_matrix(y, y_pred, labels=[0, 1])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Not True", "True"])
    disp.plot()
    plt.title(f"Confusion Matrix: Logistic Regression ({orientation} vs Not)")
    plt.show()


In [ ]:
X = data.filter(like="time")
spike_counts_per_feature = X.sum(axis=0)
spike_counts_per_feature = spike_counts_per_feature.sort_values(ascending=False)
print(spike_counts_per_feature.head(10))

In [ ]:
X = data.filter(like="time")
data["total_spikes"] = X.sum(axis=1)
data.groupby("labels.orientation")["total_spikes"].mean().sort_values(ascending=False)


### Other methods used _(if applicable)_

_(If this is a preprocessing step to prepare your data for regression or classification models, you should put this subsection before your explanation for the regression or classification models.)_

_(What method did you use otherwise? Why did you choose to use them? What questions would you answer with them? How would you evaluate the results? What cross-validation method did you use when applicable?)_

In [ ]:
# you may add some code here to show how the method works in principle

# you may add some code here to show how the model works in principle

## Results

_(What did you find when you carried out your methods? Some of your code related to
presenting results/figures/data may be replicated from the methods section or may only be present in
this section. All of the plots that you plan on using for your presentation should be present in this
section)_

### classification results
_(What are you trying to do here?)_

In [ ]:
# how did you do it

plt.figure(figsize=(25, 20))
plot_tree(first_tree, feature_names=predictors.columns, class_names= clf_tree.classes_, filled=True)

In [ ]:
accuracies_first = cross_val_score(first_tree, predictors, orientation, cv=5)

print(np.mean(accuracies_simple))

The average accuracy is quite poor, at 0.13174. Since there are 8 leaves, we would expect an accuracy of 0.125 when just randomly selecting, which shows our model is barely effective. Therefore, I would like to pursue some routes to see if I can improve upon this model and get a better accuracy. One thing I suspect is that there are too many columns in use. 400 columns is quite a lot, so I want to see if there are any columns that are all 1s or all 0s and are not telling us anything, they are just making the model more cluttered.

In [ ]:
plt.figure(figsize=(25, 20))
plot_tree(second_tree, feature_names=predictors.columns, class_names= second_tree.classes_, filled=True)

In [ ]:
accuracies2 = cross_val_score(second_tree, predictors_filtered, orientation, cv=5)

print(np.mean(accuracies2))

This still seemingly is not helping. Therefore, I will try to combine columns together by increments of 25. So, for each row, the sum of 25 continuous cells will be added up, then the next 25 continuous cells will be added up. I will keep the all 0s or all 1s columns, as that should still be relevant when I am combining the columns like this. This dataset with 400 will then be moved into a dataframe with 16 columns, and the same number of rows.

In [ ]:
plt.figure(figsize=(25, 20))
plot_tree(third_tree, feature_names=predictors.columns, class_names= third_tree.classes_, filled=True)

In [ ]:
accuracies3 = cross_val_score(third_tree, X_grouped, orientation, cv=5)

print(np.mean(accuracies3))

_(How do you interpret what you see?)_

_(What are you doing next?)_

In [ ]:
# how did you do it (etc. etc.)

### regression results
_(What are you trying to do here?)_

In [ ]:
# how did you do it

_(How do you interpret what you see?)_

_(What are you doing next?)_

In [ ]:
# how did you do it (etc. etc.)

## Discussion and Conclusion

_(What did you learn from your results? What obstacles did you run into? What would you do differently next time? Clearly provide quantitative answers to your question(s)?  At least one of your questions should be answered with numbers.  That is, it is not sufficient to answer "yes" or "no", but rather to say something quantitative such as variable 1 increased roughly 10% for every 1 year increase in variable 2.)_

### discussion on the classification results

### discussion on the regression results

### conclusion and future steps

## Author contribution

_(Please describe the contribution of each member of group)._

## References

_(List the source(s) for any data and/or literature cited in your project.  Ideally, this should be formatted using a formal citation format (MLA or APA or other, your choice!).   Multiple free online citation generators are available such as <a href="http://www.easybib.com/style">http://www.easybib.com/style</a>. **Important:** if you use **any** code that you find on the internet for your project you **must** cite it or you risk losing most/all of the points for you project.)_



    Freiwald, W. A., & Tsao, D. Y. (2010). Functional compartmentalization and viewpoint generalization within the macaque face-processing system. Science, 330(6005), 845-851.

    Meyers, E. M., Borzello, M., Freiwald, W. A., & Tsao, D. (2015). Intelligent information loss: The coding of facial identity, head pose, and non-face information in the macaque face patch system. Journal of Neuroscience, 35(18).
